In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Start

In [29]:
import random


from collections import Counter
from collections import defaultdict
from engine.player import Player
from engine.character import Character
from engine.game_manager import GameManager

# Game Manager

In [ ]:
game = GameManager(verbose=True)

## setup_game

In [30]:
game = GameManager(verbose=True)

In [41]:
for i, role_name in enumerate(game.role_pool):
    # 这里会实例化你之前定义的 Player 类
    player = Player(player_id=i)
    character = Character(role_name, player)
    game.characters.append(character)
    game.id_to_character[i] = character
    game.role_to_id[role_name].add(i)
game._initialize_beliefs()
game.public_info = game.get_game_public_info()

### _initialize_beliefs

In [31]:
game = GameManager(verbose=True)
for i, role_name in enumerate(game.role_pool):
    # 这里会实例化你之前定义的 Player 类
    player = Player(player_id=i)
    character = Character(role_name, player)
    game.characters.append(character)
    game.id_to_character[i] = character
    game.role_to_id[role_name].add(i)

In [32]:
players_count = len(game.role_pool)
all_role_counts = Counter(game.role_pool)
for observer in game.characters:
    ob_player = observer.player
    for target in game.characters:
        tar_player = target.player
        if ob_player.player_id == tar_player.player_id:
            # 对自己的认知：100% 确定
            ob_player.beliefs[tar_player.player_id] = {role: (1.0 if role == observer.role_name else 0.0) for role in all_role_counts}
            continue

        probs = {role: 0.0 for role in all_role_counts}
        if observer.role_name == 'Wolf':
            if target.role_name == 'Wolf':
                # 狼人认队友：100% 确定是狼
                probs['Wolf'] = 1.0
            else:
                # 狼人看其他人：模糊处理（知道不是狼，但不知道具体是哪个好人）
                good_role_count = players_count-all_role_counts['Wolf']
                for role in all_role_counts:
                    if role == 'Wolf':
                        continue
                    probs[role] = all_role_counts[role]/good_role_count
        else:
            # 好人看其他人：完全模糊（除了自己，剩下11个人都有可能是任何角色）
            # 假设好人视角下的剩余人数分配
            remaining_players_count = players_count-1
            remaining_role_count = all_role_counts.copy()
            remaining_role_count[observer.role_name] -= 1
            for role in all_role_counts:
                probs[role] = remaining_role_count[role]/remaining_players_count
        ob_player.beliefs[tar_player.player_id] = probs

In [36]:
game.characters[0].player.beliefs[4]

{'Wolf': 0.0,
 'Villager': 0.5,
 'Seer': 0.125,
 'Witch': 0.125,
 'Hunter': 0.125,
 'Idiot': 0.125}

In [37]:
game.characters[4].player.beliefs[0]

{'Wolf': 0.36363636363636365,
 'Villager': 0.2727272727272727,
 'Seer': 0.09090909090909091,
 'Witch': 0.09090909090909091,
 'Hunter': 0.09090909090909091,
 'Idiot': 0.09090909090909091}

In [38]:
game.characters[8].player.beliefs[0]

{'Wolf': 0.36363636363636365,
 'Villager': 0.36363636363636365,
 'Seer': 0.0,
 'Witch': 0.09090909090909091,
 'Hunter': 0.09090909090909091,
 'Idiot': 0.09090909090909091}

### get_game_public_info

In [39]:
game = GameManager(verbose=True)
for i, role_name in enumerate(game.role_pool):
    # 这里会实例化你之前定义的 Player 类
    player = Player(player_id=i)
    character = Character(role_name, player)
    game.characters.append(character)
    game.id_to_character[i] = character
    game.role_to_id[role_name].add(i)
game._initialize_beliefs()

In [40]:
{
    "alive_player_ids": {c.player.player_id for c in game.characters if c.player.is_alive},
    "gold_water_claims": {c.player.player_id: c.player.gold_water_claims for c in game.characters if c.player.is_alive},
    "silver_water_claims": {c.player.player_id: c.player.silver_water_claims for c in game.characters if c.player.is_alive},
    "public_claims": {c.player.player_id: c.player.public_role_claim for c in game.characters},
    "sheriff_id": next((c.player.player_id for c in game.characters if c.player.is_sheriff), None),
    "current_day": game.current_day,
    # 这里还可以加入历史投票记录、已确认死亡名单等
}

{'alive_player_ids': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
 'gold_water_claims': {0: {},
  1: {},
  2: {},
  3: {},
  4: {},
  5: {},
  6: {},
  7: {},
  8: {},
  9: {},
  10: {},
  11: {}},
 'silver_water_claims': {0: {},
  1: {},
  2: {},
  3: {},
  4: {},
  5: {},
  6: {},
  7: {},
  8: {},
  9: {},
  10: {},
  11: {}},
 'public_claims': {0: None,
  1: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None},
 'sheriff_id': None,
 'current_day': 0}

## run_simulation

In [104]:
game = GameManager(verbose=True)
game.setup_game()
game.run_simulation()

--- 第 0 晚开始 ---
狼人请行动
狼人选择杀害10号玩家
预言家请行动
预言家选择查验0号玩家
女巫请行动
女巫选择救10号玩家
--- 第 1 晚开始 ---
狼人请行动
狼人选择杀害10号玩家
预言家请行动
预言家选择查验1号玩家
女巫请行动
女巫选择毒死0号玩家
--- 第 2 晚开始 ---
狼人请行动
狼人选择杀害9号玩家
预言家请行动
预言家选择查验2号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 3 晚开始 ---
狼人请行动
狼人选择杀害10号玩家
预言家请行动
预言家选择查验3号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 4 晚开始 ---
狼人请行动
狼人选择杀害8号玩家
预言家请行动
预言家选择查验4号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 5 晚开始 ---
狼人请行动
狼人选择杀害8号玩家
预言家请行动
预言家选择查验5号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 6 晚开始 ---
狼人请行动
狼人选择杀害11号玩家
预言家请行动
预言家选择查验6号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 7 晚开始 ---
狼人请行动
狼人选择杀害5号玩家
预言家请行动
预言家选择查验7号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 8 晚开始 ---
狼人请行动
狼人选择杀害9号玩家
预言家请行动
预言家选择查验9号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 9 晚开始 ---
狼人请行动
狼人选择杀害8号玩家
预言家请行动
预言家选择查验10号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 10 晚开始 ---
狼人请行动
狼人选择杀害9号玩家
预言家请行动
预言家选择查验11号玩家
女巫请行动
女巫选择不救人也不毒死
--- 第 11 晚开始 ---
狼人请行动
狼人选择杀害8号玩家
预言家请行动


ValueError: max() iterable argument is empty

### check_game_over

In [ ]:
game = GameManager(verbose=True)
game.setup_game()

In [43]:
alive_ids = game.public_info['alive_player_ids']
wolf_ids = game.role_to_id['Wolf']
villager_ids = game.role_to_id['Villager']
good_ids = set()
for role, id in game.role_to_id.items():
    if (role != 'Wolf') and (role != 'Villager'):
        good_ids = good_ids.union(id)

In [47]:
alive_ids = {1,2}
alive_ids = {4,8}

In [48]:
if not alive_ids.intersection(wolf_ids):
    game.winner = 'Villager'
    if game.verbose:
        print("所有狼人死亡，好人获胜")
    print(True)
if not alive_ids.intersection(villager_ids):
    game.winner = 'Wolf'
    if game.verbose:
        print("所有平民死亡，狼人获胜")
    print(True)
if not alive_ids.intersection(good_ids):
    game.winner = 'Wolf'
    if game.verbose:
        print("所有神职死亡，狼人获胜")
    print(True)
print(False)

所有狼人死亡，好人获胜
True
False


### night_phase

#### wolf

In [92]:
game = GameManager(verbose=True)
game.setup_game()
game.check_game_over()

False

In [93]:
target_ids = []
wolves = [game.characters[id] for id in game.role_to_id['Wolf'] if id in game.public_info['alive_player_ids']]
for wolf in wolves:
    target_id = wolf.role.handle_night_action(wolf, game.public_info)
    target_ids.append(target_id)
target_ids

[4, 7, 10, 4]

In [94]:
wolves_target = max(set(target_ids), key=target_ids.count)
game.private_info[game.current_day]['wolves_target'] = wolves_target
game.private_info

defaultdict(dict, {0: {'wolves_target': 4}})

#### seer

In [95]:
seer = next((c for c in game.characters if c.player.is_alive and c.role_name == "Seer"), None)
seer      

In [96]:
seer_target = seer.role.handle_night_action(seer, game.public_info)
seer_target

0

In [97]:
game.private_info[game.current_day]['seer_target'] = seer_target
game.private_info

defaultdict(dict, {0: {'wolves_target': 4, 'seer_target': 0}})

In [98]:
target_char = game.characters[seer_target]
is_good = target_char.role_name != 'Wolf'
seer.role.handle_check_result(seer, seer_target, is_good)

In [99]:
seer.player.beliefs[seer_target]

{'Wolf': 1.0,
 'Villager': 0.0,
 'Seer': 0.0,
 'Witch': 0.0,
 'Hunter': 0.0,
 'Idiot': 0.0}

#### witch

In [100]:
witch = next((c for c in game.characters if c.player.is_alive and c.role_name == "Witch"), None)

In [101]:
witch_heal_target, witch_poison_target = witch.role.handle_night_action(witch, game.public_info, game.private_info[game.current_day])
[witch_heal_target, witch_poison_target]

[4, None]